In [2]:
#!pip install gensim
import gensim
import pandas as pd
import nltk
import re
from collections import defaultdict

In [ ]:
import tarfile
tar = tarfile.open("araneum_none_fasttextcbow_300_5_2018.tgz", "r:gz")
for member in tar.getmembers():
     f = tar.extractfile(member)
     if f is not None:
         content = f.read()

tar.extractall()

In [4]:
df = pd.read_csv('result.tar.gz', compression='gzip', sep='\t', error_bad_lines=False)

b'Skipping line 37039: expected 12 fields, saw 15\nSkipping line 45611: expected 12 fields, saw 15\n'
b'Skipping line 354339: expected 12 fields, saw 18\nSkipping line 360804: expected 12 fields, saw 102\nSkipping line 360819: expected 12 fields, saw 78\nSkipping line 379126: expected 12 fields, saw 14\nSkipping line 444446: expected 12 fields, saw 16\nSkipping line 449079: expected 12 fields, saw 16\nSkipping line 450250: expected 12 fields, saw 15\n'
b'Skipping line 481374: expected 12 fields, saw 18\nSkipping line 510619: expected 12 fields, saw 20\nSkipping line 512271: expected 12 fields, saw 14\nSkipping line 514317: expected 12 fields, saw 15\nSkipping line 519707: expected 12 fields, saw 40\n'
b'Skipping line 538495: expected 12 fields, saw 13\n'
b'Skipping line 765901: expected 12 fields, saw 56\nSkipping line 773310: expected 12 fields, saw 43\n'
b'Skipping line 805899: expected 12 fields, saw 13\nSkipping line 868492: expected 12 fields, saw 21\nSkipping line 871801: expecte

In [5]:
df.dropna(inplace=True)

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df.rename(columns={'result.tsv': 'text'}, inplace=True)

In [8]:
df = df[['text', 'odd', 'tip_name', 'bookmaker_name', 'match_date_time', 'league_name_in_russian',
    'team_of_season_1_name_in_russian', 'team_of_season_2_name_in_russian']]

In [9]:
def tokenize(text):
    text = text.replace('\xa0', '')
    text = text.replace('«', '')
    text = text.replace('»', '')
    text = re.findall(r'[А-Яа-яA-Za-z0-9]+', text)
    text = [word.lower() for word in text]
    return text

def ngram_range(text, min_gram=1, max_gram=3):
    text = tokenize(text)
    if len(text) == 1 or len(text) == 0:
        return text
    grams = nltk.everygrams(text, min_gram, max_gram)    
    grams = [' '.join(gram) for gram in grams]
    grams = set(grams)
    return grams


In [10]:
df['grams'] = df.text.apply(ngram_range)

In [29]:
def direct_match(df, gram_col, col_to_compare):
    result = []
    for x, y in zip(df[gram_col], df[col_to_compare]):
        inter = x.intersection(ngram_range(y)) # частичное совпадение
        #inter = y.lower() in x # полное совпадение
        result.append(bool(inter))
        new_col = col_to_compare + '_found'
    df[new_col] = result

In [47]:
direct_match(df, 'grams', 'team_of_season_1_name_in_russian')
df['team_of_season_1_name_in_russian_found'].value_counts()

True     1733
False     151
Name: team_of_season_1_name_in_russian_found, dtype: int64

In [48]:
direct_match(df, 'grams', 'team_of_season_2_name_in_russian')
#df.loc[~df['team_of_season_2_name_in_russian_found']].head()
df['team_of_season_2_name_in_russian_found'].value_counts()

True     1667
False     217
Name: team_of_season_2_name_in_russian_found, dtype: int64

In [49]:
direct_match(df, 'grams', 'league_name_in_russian')
df['league_name_in_russian_found'].value_counts()

True     1037
False     847
Name: league_name_in_russian_found, dtype: int64

In [50]:
direct_match(df, 'grams', 'bookmaker_name')
df['bookmaker_name_found'].value_counts()

False    1461
True      423
Name: bookmaker_name_found, dtype: int64

In [55]:
def direct_match_odd(df, gram_col, col_to_compare):
    result = []
    for x, y in zip(df[gram_col], df[col_to_compare]):
        inter = y in x # полное совпадение
        result.append(bool(inter))
        new_col = col_to_compare + '_found'
    df[new_col] = result

In [56]:
direct_match_odd(df, 'grams', 'odd')
#df.loc[~df['odd_found']].head()
df['odd_found'].value_counts()

False    1864
True       20
Name: odd_found, dtype: int64

In [5]:
model = gensim.models.KeyedVectors.load("araneum_none_fasttextcbow_300_5_2018.model")

FileNotFoundError: [Errno 2] No such file or directory: 'araneum_none_fasttextcbow_300_5_2018.model'

In [15]:
df['model'] = df.tip_name.apply(lambda x: model[x])

In [17]:
import numpy as np

In [19]:
from itertools import combinations

In [24]:
from scipy.spatial.distance import cosine

In [39]:
result = {}
for i, j in combinations(df.sample(100).index.values, 2):
    result[(i, j)] = cosine(df.model[i].reshape(-1, 1), df.model[j].reshape(-1, 1))

In [40]:
import operator

In [55]:
sorted(result.items(), key=operator.itemgetter(1))[:20]

[((494670, 481915), 0.0),
 ((494670, 491097), 0.0),
 ((494670, 493995), 0.0),
 ((294516, 296312), 0.0),
 ((294516, 305983), 0.0),
 ((294516, 306415), 0.0),
 ((294516, 304519), 0.0),
 ((294516, 308303), 0.0),
 ((294516, 305163), 0.0),
 ((294516, 292828), 0.0),
 ((294516, 309652), 0.0),
 ((294516, 304408), 0.0),
 ((358693, 396489), 0.0),
 ((358693, 413955), 0.0),
 ((358693, 379545), 0.0),
 ((358693, 381780), 0.0),
 ((358693, 429436), 0.0),
 ((358693, 430003), 0.0),
 ((358693, 357505), 0.0),
 ((358693, 406447), 0.0)]

In [54]:
df.tip_name[868315]

'Х2 и обе забьют'

In [53]:
df.tip_name[242994]

'Обе забьют'